In [1]:
import pandas as pd
import math
import numpy as np

### Understanding Feature Selection by Entropy

In [2]:
df = pd.read_csv('car_dataset.csv')
df.columns = [col.lower() for col in df.columns]
df.sample(5)

,type,color,tyre,doors,label
7,minivan,red,black,4,0
3,suv,green,white,4,1
2,car,blue,white,2,1
11,car,blue,black,4,0
6,car,green,white,4,0


In [3]:
def gen_num_case_dict(df):
  feq = df['label'].value_counts().to_dict()
  return feq

num_case_dict = gen_num_case_dict(df)
num_case_dict

{0: 9, 1: 5}

In [4]:
def cal_entropy(num_case_dict):
  total = 0
  for k, d in num_case_dict.items():
    total += d

  entropy_sum = 0
  for k, d in num_case_dict.items():
    entropy_sum += - (d/total) * math.log2(d/total) # Tell how data is separate | greater = more equality

  return entropy_sum

cal_entropy(num_case_dict)

0.9402859586706311

In [5]:
df_red = df[ df['color'] == 'red']

display(df_red)

num_case_dict_red = gen_num_case_dict(df_red)

print(num_case_dict_red)

entropy_red = cal_entropy(num_case_dict_red)

print('entropy_red', entropy_red) # entropy 1 is not a good feature for classification

entropy_red_weighted = (len(df_red)/len(df)) * entropy_red

# greater is better for spliting
entropy_red_weighted

,type,color,tyre,doors,label
0,suv,red,white,2,1
4,car,red,black,2,1
7,minivan,red,black,4,0
10,suv,red,black,2,0


{1: 2, 0: 2}
entropy_red 1.0


0.2857142857142857

In [6]:
# calculate the entropy gain from a decision tree split using a particular feature on a particular df
def cal_entropy_split_gain(df, split_feature):
  parent = cal_entropy(gen_num_case_dict(df))

  cris = df[split_feature].value_counts().index.to_list()

  entropy_gain = 0

  for cri in cris:
    splited = df[ df[split_feature] == cri ]
    feq = gen_num_case_dict(splited)
    entropy = cal_entropy(feq)

    entropy_gain += (len(splited)/len(df)) * entropy

  return parent - entropy_gain

print("color:", cal_entropy_split_gain(df, 'color'))
print("type:", cal_entropy_split_gain(df, 'type'))
print("door", cal_entropy_split_gain(df, 'doors'))
print("tyre:", cal_entropy_split_gain(df, 'tyre'))

# Seeing that type provide Best split from maximum entropy gain

color: 0.02922256565895487
type: 0.19996253177061118
door 0.15183550136234159
tyre: 0.04812703040826949


In [7]:
# Given the df and available features for conducting split, find the feature that produces maximum entropy gain.
def choose_best_split(df, available_feature_array):
  mx = int(-1e9)
  feature_with_max_entropy_gain = None

  for feature in available_feature_array:
    temp = cal_entropy_split_gain(df, feature)
    if temp > mx:
      mx = temp
      feature_with_max_entropy_gain = feature

  return feature_with_max_entropy_gain

columns_list = df.columns.tolist()
columns_list.remove('label')
best_split_feature = choose_best_split(df, columns_list)
best_split_feature

'type'

In [8]:
# Generate a dictionary containing the split key and resulting df due to the split key and split feature.
def gen_feature_split_result(df, split_feature):
  sub_elems = df[split_feature].unique()

  df_split_dict = {}

  for sub in sub_elems:
    df_split_dict[sub] = df.loc[df[split_feature] == sub]

  return df_split_dict

df_split_dict = gen_feature_split_result(df, best_split_feature)
df_split_dict.keys()

dict_keys(['suv', 'car', 'minivan'])

In [9]:
df_split_dict['suv']

,type,color,tyre,doors,label
0,suv,red,white,2,1
3,suv,green,white,4,1
8,suv,green,black,4,0
9,suv,blue,black,2,0
10,suv,red,black,2,0
12,suv,green,black,2,0


In [10]:
df_split_dict['minivan']

,type,color,tyre,doors,label
5,minivan,blue,white,4,0
7,minivan,red,black,4,0
13,minivan,green,white,4,0


In [11]:
df_split_dict['car']

,type,color,tyre,doors,label
1,car,green,black,2,1
2,car,blue,white,2,1
4,car,red,black,2,1
6,car,green,white,4,0
11,car,blue,black,4,0


### Create Tree Stucture

In [12]:
class Node:
  def __init__(self, df, parent, parent_edge, parent_split_feature, available_features, pred):
    self.df = df
    self.parent = parent
    self.parent_edge = parent_edge
    self.children = {}
    self.split_feature = None
    self.available_features = available_features
    self.explored = False
    self.predicted_label = pred
    if parent is not None:
      parent.split_feature = parent_split_feature

  def add_child(self, split_key, new_node):
    self.children[split_key] = new_node

In [13]:
class TreeGraph:
  def __init__(self, df, available_features):
    self.root = Node(df, None, None, None, available_features, None)
    self.nodes = []
    self.nodes.append(self.root)

  def add_node(self, df, parent_node, parent_edge, parent_split_feature, available_features, pred):
    new_node = Node(df, parent_node, parent_edge, parent_split_feature, available_features, pred)
    self.nodes.append(new_node)
    if parent_node is not None:
      parent_node.add_child(  parent_edge, new_node )

  def gen_decision_tree(self):
    for id, node in enumerate(self.nodes):

      # This node already provide the answer
      if node.predicted_label is not None:
        continue

      available_features = node.df.columns.to_list()
      available_features.remove('label')

      # Here is no features left to split
      if len(available_features) == 0:
        respones = gen_num_case_dict(node.df)

        mx, pred = -1e9, None
        for k, d in respones.items():
          if mx <= d:
            mx = d
            pred = k

        node.predicted_label = pred
        continue

      best_split = choose_best_split(node.df, available_features)
      df_split_dict = gen_feature_split_result(node.df, best_split)
      available_features.remove(best_split)

      for edge, sub_df in df_split_dict.items():
        respones = gen_num_case_dict(sub_df)

        if len(respones) == 1:
          self.add_node(sub_df, node, str(edge), best_split, available_features, list(respones.keys())[0])
          continue

        self.add_node(sub_df, node, str(edge), best_split, available_features, None)

In [14]:
columns_list = df.columns.tolist()
columns_list.remove('label')
g = TreeGraph(df, columns_list)
g.gen_decision_tree()

print("num of node:", len(g.nodes), "\nroot feature:", g.root.split_feature)

num of node: 8 
root feature: type


In [15]:
def generate_prediction(df_prediction, treeGraph, printing=False):
  for i, query in zip(df_prediction.index, df_prediction.iloc):
    curr = treeGraph.root
    # display(query)
    while curr.predicted_label is None:
      edge = query[curr.split_feature]
      if printing:
        print(curr.split_feature, edge, "->", end=" ")
      
      if str(edge) in list(curr.children.keys()):
        curr = curr.children[str(edge)]
      else:
        if printing:
          print(sorted(curr.children.keys()))
        mn = 1e9
        mx = -1e9
        mx_id = None
        data = sorted(curr.children.items())
        for k, j in data:
          if len(j.df) < mn:
            mn = len(j.df)
          if len(j.df) > mx:
            mx = len(j.df)
            mx_id = k
        if mx == mn:
          curr = data[len(data)//2][1]
        else:
          curr = curr.children[str(mx_id)]
        break
    
    if printing:
      print(curr.predicted_label)
    df_prediction.loc[i, 'prediction'] = curr.predicted_label

  return df_prediction

In [16]:
df_prediction = df.copy()
df_prediction['prediction'] = None

pd.options.mode.copy_on_write = True
prediction_result_df = generate_prediction(df_prediction, g, printing=True)

type suv -> tyre white -> 1
type car -> doors 2 -> 1
type car -> doors 2 -> 1
type suv -> tyre white -> 1
type car -> doors 2 -> 1
type minivan -> 0
type car -> doors 4 -> 0
type minivan -> 0
type suv -> tyre black -> 0
type suv -> tyre black -> 0
type suv -> tyre black -> 0
type car -> doors 4 -> 0
type suv -> tyre black -> 0
type minivan -> 0


In [17]:
def prediction_accuracy(prediction_result_df):
  correct = 0
  for index, row in prediction_result_df.iterrows():
    if row['label'] == row['prediction']:
      correct = correct + 1
  accuracy = correct / len(prediction_result_df)
  return accuracy

prediction_accuracy_result = prediction_accuracy(prediction_result_df)
prediction_accuracy_result

1.0

### Test with Proper Dataset

In [18]:
df_car = pd.read_csv('car.data', sep=',')
df_car.head()

,buying,maintainance,doors,person,luggage_boot,safety,label
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [19]:
from sklearn.model_selection import train_test_split
df_car_train, df_car_test = train_test_split(df_car, test_size=0.2, random_state=42)

In [20]:
columns_list = df_car.columns.tolist()
columns_list.remove('label')
g_car = TreeGraph(df_car_train, columns_list)
g_car.gen_decision_tree()
len(g_car.nodes)

349

In [21]:
g_car.root.split_feature

'safety'

In [22]:
df_car_prediction = df_car_test.copy()
df_car_prediction['prediction'] = None
df_car_prediction_result = generate_prediction(df_car_prediction, g_car)
df_car_prediction_result.sample(5)

,buying,maintainance,doors,person,luggage_boot,safety,label,prediction
966,med,vhigh,5more,more,med,low,unacc,unacc
730,high,med,5more,2,small,med,unacc,unacc
1431,low,high,3,2,small,low,unacc,unacc
710,high,med,4,2,big,high,unacc,unacc
1270,med,low,5more,2,small,med,unacc,unacc


In [23]:
prediction_accuracy_result = prediction_accuracy(df_car_prediction_result)
prediction_accuracy_result

0.9190751445086706